In [1]:
from games_setup import *
import SBMLLint.common.constants as cn
from SBMLLint.common.reaction import Reaction
from SBMLLint.common.stoichiometry_matrix import StoichiometryMatrix
from SBMLLint.games.som import SOM
from SBMLLint.games.mesgraph import MESGraph
from SBMLLint.games.games_pp import GAMES_PP, SOMStoichiometry, SOMReaction, TOLERANCE
from SBMLLint.games.games_report import GAMESReport, SimplifiedReaction
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
from scipy.linalg import lu, inv

Current Directory: /Users/woosubs/Desktop/ModelEngineering/SBMLLint/SBMLLint/notebooks


In [2]:
file = "BIOMD0000000167_url.xml"
simple = SimpleSBML()
simple.initialize(os.path.join(cn.BIOMODELS_DIR, file))
m = GAMES_PP(simple)
s = StoichiometryMatrix(simple)

In [3]:
res = m.analyze(simple_games=False, error_details=True)

Model analyzed...
Type I error:  []
Type II error:  []
Canceling error:  []
Echelon error:  [statPhosphorylation:  -> {species_test}]
Type III error:  [PstatDimerisation: 2.00 {Pstat_sol} -> 2.00 {Pstat_nuc=stat_nuc=stat_sol}] 

At least one error found.



In [4]:
gr = GAMESReport(m)

In [20]:
rep = gr.reportEchelonError(echelon_errors=m.echelon_errors, explain_details=False)
print(rep[0])
print(rep[1])


We detected a mass imbalance
:  -> species_test

from the following isolation set.

1. PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol
2. statPhosphorylation: stat_sol -> Pstat_sol + species_test
3. PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc
4. PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc
5. stat_export: stat_sol -> stat_nuc
6. statDephosphorylation: Pstat_nuc -> stat_nuc


**********************************************************************


[3]


In [6]:
m.reactions

[PstatDimerisation: 2.00 Pstat_sol -> PstatDimer_sol; (Kf_PstatDimerisation * pow(Pstat_sol, 2) + -(Kr_PstatDimerisation * PstatDimer_sol)) * sol,
 statDephosphorylation: Pstat_nuc -> stat_nuc; Kcat_dephos * statPhosphatase_nuc * Pstat_nuc * (1 / (Km_dephos + Pstat_nuc)) * nuc,
 statPhosphorylation: stat_sol -> Pstat_sol + species_test; Kcat_phos * statKinase_sol * stat_sol * (1 / (Km_phos + stat_sol)) * sol,
 PstatDimerisationNuc: 2.00 Pstat_nuc -> PstatDimer_nuc; (Kf_PstatDimerisation * pow(Pstat_nuc, 2) + -(Kr_PstatDimerisation * PstatDimer_nuc)) * nuc,
 PstatDimer__import: PstatDimer_sol -> PstatDimer_nuc; PstatDimer_impMax * PstatDimer_sol * (1 / (Kpsd_imp + PstatDimer_sol)) * nm,
 stat_export: stat_sol -> stat_nuc; nuc * stat_expMax * stat_nuc * (1 / (Ks_exp + stat_nuc)) * nm,
 stat_import: stat_sol -> stat_nuc; nuc * stat_impMax * stat_sol * (1 / (Ks_imp + stat_sol)) * nm]

In [7]:
m.som_reactions_lu

[statPhosphorylation: {Pstat_nuc=stat_nuc=stat_sol} -> {Pstat_sol} + {species_test},
 PstatDimerisation: 2.00 {Pstat_sol} -> {PstatDimer_nuc=PstatDimer_sol},
 PstatDimerisationNuc: 2.00 {Pstat_nuc=stat_nuc=stat_sol} -> {PstatDimer_nuc=PstatDimer_sol}]

In [8]:
m.som_reactions_lu
reaction_labels = [r.label for r in m.som_reactions_lu]
print(reaction_labels)
species_names = [s.identifier for s in list(m.nodes)]
print(species_names)

['statPhosphorylation', 'PstatDimerisation', 'PstatDimerisationNuc']
['{Pstat_sol}', '{species_test}', '{PstatDimer_nuc=PstatDimer_sol}', '{Pstat_nuc=stat_nuc=stat_sol}']


In [9]:
### issues (they might be conected)
# 1. list duplication in m.som_reactions_lu
# 2. slicing (assigning values) to matrix in m.getStoichioemtryMatrix. 
# try to change it to .loc assignment

In [10]:
m.getStoichiometryMatrix(m.som_reactions_lu, list(m.nodes), som=True)

,statPhosphorylation,PstatDimerisation,PstatDimerisationNuc
{Pstat_sol},1.0,-2.0,0.0
{species_test},1.0,0.0,0.0
{PstatDimer_nuc=PstatDimer_sol},0.0,1.0,1.0
{Pstat_nuc=stat_nuc=stat_sol},-1.0,0.0,-2.0


In [11]:
k = m.getStoichiometryMatrix(m.som_reactions_lu, list(m.nodes), som=True)
k.loc["{Pstat_sol}", "statPhosphorylation"] = 2.33
k

,statPhosphorylation,PstatDimerisation,PstatDimerisationNuc
{Pstat_sol},2.33,-2.0,0.0
{species_test},1.00,0.0,0.0
{PstatDimer_nuc=PstatDimer_sol},0.00,1.0,1.0
{Pstat_nuc=stat_nuc=stat_sol},-1.00,0.0,-2.0


In [12]:
mat_df = m.som_stoichiometry_matrix
reactions = []
for reaction_name in mat_df.columns:
  reaction_elements = mat_df[reaction_name]
  reactants = [SOMStoichiometry(
      m.getNode(som_label),
      np.round(abs(reaction_elements[som_label]), 3)
      ) \
      for som_label in reaction_elements.index \
      if reaction_elements[som_label]<TOLERANCE*(-1)]
#   products = [SOMStoichiometry(
#       m.getNode(som_label),
#       np.round(abs(reaction_elements[som_label]), 3)
#       ) \
#       for som_label in reaction_elements.index \
#       if reaction_elements[som_label]>TOLERANCE]
#   reactions.append(SOMReaction(
#       reactants=reactants,
#       products=products,
#       label=reaction_name
#       ))